In [1]:
import itertools
import numpy as np
from multiprocessing.dummy import Pool
import os.path
import mpu
import time
import sys
sys.path.append('../')
from card import Card
import pickle

In [3]:
np.array(list(itertools.combinations(range(52),7))).shape

(133784560, 7)

In [2]:
if os.path.isfile('SevenCardHands_24.npy'):
    SevenCardHands = np.load('SevenCardHands_24.npy')
else:
    SevenCardHands = np.array(list(itertools.combinations(range(24),7)))
    np.save('SevenCardHands_24.npy', SevenCardHands)
print(SevenCardHands.shape)

(346104, 7)


In [3]:
if os.path.isfile('SixCardHands.npy'):
    SixCardHands = np.load('SixCardHands_24.npy')
else:
    SixCardHands = np.array(list(itertools.combinations(range(28),6)))
    np.save('SixCardHands_24.npy', SixCardHands)
print(SixCardHands.shape)

(376740, 6)


In [4]:
if os.path.isfile('FiveCardHands_24.npy'):
    FiveCardHands = np.load('FiveCardHands_24.npy')
else:
    FiveCardHands = np.array(list(itertools.combinations(range(24),5)))
    np.save('FiveCardHands_24.npy', FiveCardHands)
print(FiveCardHands.shape)

(42504, 5)


In [5]:
if os.path.isfile('TwoCardHands_24.npy'):
    TwoCardHands = np.load('TwoCardHands_24.npy')
else:
    TwoCardHands = np.array(list(itertools.combinations(range(24),2)))
    np.save('TwoCardHands_24.npy', TwoCardHands)
print(TwoCardHands.shape)

(276, 2)


In [6]:
def score_hand(hand):
    flush = hand.sum(axis=0).sum(axis=1)
    is_flush = np.max(flush) >= 5
    card_values = hand.sum(axis=0).sum(axis=0)
    count = 0
    hand_high = 4
    is_stright = False
    stright_high = 5
    for i, s in enumerate(card_values): 
        if s >= 1:
            count += 1
            hand_high = i
            if count >=5:
                stright_high = i
                is_stright = True
        else:
            count = 0
    if is_stright and is_flush:
        return 8, stright_high, hand_high    
    is_four_of_a_kind = np.isin(card_values, 4)
    if np.sum(is_four_of_a_kind):
        return 7, np.argwhere(is_four_of_a_kind==True)[0][0], hand_high
    is_three_of_a_kind = np.isin(card_values, 3)
    is_pair = np.isin(card_values, 2)
    if np.sum(is_pair) and np.sum(is_three_of_a_kind):
        return 6, np.argwhere(is_three_of_a_kind==True)[0][0], hand_high
    if is_flush:
        return 5, np.argwhere(hand.sum(axis=0)[np.argmax(flush)]==1)[-1][0], hand_high
    if is_stright:
        return 4, stright_high, hand_high
    if np.sum(is_three_of_a_kind):
        return 3, np.argwhere(is_three_of_a_kind==True)[0][0], hand_high
    if np.sum(is_pair) >= 2:
        return 2, np.argwhere(is_pair==True)[1][0], hand_high
    if np.sum(is_pair):
        return 1, np.argwhere(is_pair==True)[0][0], hand_high
    return 0, 0, hand_high

In [7]:
card_hands = [TwoCardHands, FiveCardHands, SixCardHands, SevenCardHands]
for j, hand_card in enumerate([2, 5, 6, 7]):
    def score_hands(index):
        hand = np.array(card_hands[j][int(index)], dtype=np.int8)
        histo = np.zeros((9))
        deck = np.arange(24)
        deck = np.delete(deck, hand)
        for i in range(iters):
            np.random.shuffle(deck)
            cards = np.zeros((num_cards,24))
            for k, c in enumerate(hand):
                cards[k,c] = 1
            for k in range(len(hand),num_cards):
                cards[k,deck[k]] = 1
            cards = cards.reshape(num_cards,4,6)
            histo[score_hand(cards)[0:1]] += 1
        score = 0
        for i in range(9):
            score += (i+1)*histo[i]/iters
        scores[hand.tobytes()] = score

    scores = {}
    num_cards = 7
    iters_ = [10, 100, 1000]
    for iter in iters_:
        iters  = iter
        p = Pool(10)
        p.map(score_hands, range(card_hands[j].shape[0]))
        p.close()
        p.join()
        mpu.io.write(str(hand_card) + '_card_values_24_'+str(iter)+'.pickle', scores)

In [ ]:
def score_hands(index):
    hand = TwoCardHands[int(index)]
    histo = np.zeros((9))
    deck = np.arange(24)
    deck = np.delete(deck, hand)
    for i in range(iters):
        np.random.shuffle(deck)
        cards = np.zeros((num_cards,24))
        for k, c in enumerate(hand):
            cards[k,c] = 1
        for k in range(len(hand),num_cards):
            cards[k,deck[k]] = 1
        cards = cards.reshape(num_cards,4,7)
        histo[score_hand(cards)[0:1]] += 1
    score = 0
    for i in range(9):
        score += (i+1)*histo[i]/iters
    scores[hand.tobytes()] = score


scores = {}
num_cards = 7
iters_ = [10, 100, 1000]
for iter in iters_:
    iters  = iter
    p = Pool(5)
    p.map(score_hands, range(TwoCardHands.shape[0]))
    p.close()
    p.join()
    mpu.io.write('2_card_values_'+str(iter)+'.pickle', scores)

In [ ]:
for i in [10,100,1000]:
    dic = mpu.io.read('2_card_values_24_' + str(i) + '.pickle')
    dic.update(mpu.io.read('5_card_values_24_' + str(i) + '.pickle'))
    dic.update(mpu.io.read('6_card_values_24_' + str(i) + '.pickle'))
    dic.update(mpu.io.read('7_card_values_24_' + str(i) + '.pickle'))
    mpu.io.write('hand_value_table_24_'+str(i)+'.pickle', dic)

In [ ]:
np.array([12,18], dtype=np.int8).tobytes()

b'\x0c\x12'

In [ ]:
SevenCardHands = np.array(list(itertools.combinations(range(24),7)))
SixCardHands = np.array(list(itertools.combinations(range(24),6)))
FiveCardHands = np.array(list(itertools.combinations(range(24),5)))
TwoCardHands = np.array(list(itertools.combinations(range(24),2)))

def norm(val, max, min):
    return (val - min) / (max - min)

value_table = mpu.io.read('hand_value_table_24_10.pickle')
normed_hand_vals = {}
two_hand_vals = []
for i in range(TwoCardHands.shape[0]):
    two_hand_vals.append(value_table[np.array(TwoCardHands[i], dtype=np.int8).tobytes()])
min = np.array(two_hand_vals).min()
max = np.array(two_hand_vals).max()

for i in range(TwoCardHands.shape[0]):
    normed_hand_vals[np.array(TwoCardHands[i], dtype=np.int8).tobytes()] = norm(value_table[np.array(TwoCardHands[i], dtype=np.int8).tobytes()], max, min)

five_hand_vals = []
for i in range(FiveCardHands.shape[0]):
    five_hand_vals.append(value_table[np.array(FiveCardHands[i], dtype=np.int8).tobytes()])
min = np.array(five_hand_vals).min()
max = np.array(five_hand_vals).max()

for i in range(FiveCardHands.shape[0]):
    normed_hand_vals[np.array(FiveCardHands[i], dtype=np.int8).tobytes()] = norm(value_table[np.array(FiveCardHands[i], dtype=np.int8).tobytes()], max, min)

six_hand_vals = []
for i in range(SixCardHands.shape[0]):
    six_hand_vals.append(value_table[np.array(SixCardHands[i], dtype=np.int8).tobytes()])
min = np.array(six_hand_vals).min()
max = np.array(six_hand_vals).max()

for i in range(SixCardHands.shape[0]):
    normed_hand_vals[np.array(SixCardHands[i], dtype=np.int8).tobytes()] = norm(value_table[np.array(SixCardHands[i], dtype=np.int8).tobytes()], max, min)

seven_hand_vals = []
for i in range(SevenCardHands.shape[0]):
    seven_hand_vals.append(value_table[np.array(SevenCardHands[i], dtype=np.int8).tobytes()])
min = np.array(seven_hand_vals).min()
max = np.array(seven_hand_vals).max()

for i in range(SevenCardHands.shape[0]):
    normed_hand_vals[np.array(SevenCardHands[i], dtype=np.int8).tobytes()] = norm(value_table[np.array(SevenCardHands[i], dtype=np.int8).tobytes()], max, min)


mpu.io.write('hand_value_table_norm_int8_24_10.pickle', normed_hand_vals)

{b'\x00\x01': 0.20588235294117638,
 b'\x00\x02': 0.2941176470588235,
 b'\x00\x03': 0.41176470588235276,
 b'\x00\x04': 0.41176470588235276,
 b'\x00\x05': 0.35294117647058826,
 b'\x00\x06': 0.7352941176470588,
 b'\x00\x07': 0.3823529411764705,
 b'\x00\x08': 0.14705882352941174,
 b'\x00\t': 0.2941176470588235,
 b'\x00\n': 0.176470588235294,
 b'\x00\x0b': 0.23529411764705874,
 b'\x00\x0c': 0.5882352941176467,
 b'\x00\r': 0.6176470588235292,
 b'\x00\x0e': 0.23529411764705874,
 b'\x00\x0f': 0.4705882352941175,
 b'\x00\x10': 0.3823529411764705,
 b'\x00\x11': 0.5,
 b'\x00\x12': 0.35294117647058826,
 b'\x00\x13': 0.35294117647058826,
 b'\x00\x14': 0.5,
 b'\x00\x15': 0.4999999999999997,
 b'\x00\x16': 0.2058823529411765,
 b'\x00\x17': 0.23529411764705874,
 b'\x01\x02': 0.5,
 b'\x01\x03': 0.5294117647058822,
 b'\x01\x04': 0.3823529411764705,
 b'\x01\x05': 0.7647058823529412,
 b'\x01\x06': 0.441176470588235,
 b'\x01\x07': 0.7058823529411763,
 b'\x01\x08': 0.2941176470588235,
 b'\x01\t': 0.382352941

In [ ]:
mpu.io.read('hand_value_table_1000.pickle')[np.array(TwoCardHands[0],dtype=np.int32).tobytes()]

4.046000000000001

In [ ]:
import numpy as np
FiveCardHands = np.array(list(itertools.combinations(range(28),5)))
FiveCardHands[0].tobytes()

b'\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00'